# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 44 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.04515


extracting tarball to tmp_2212.04515...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.04521


extracting tarball to tmp_2212.04521...

...

extracting tarball to tmp_2212.04962...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.04990


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.04962/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2212.04990...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04710-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04710) | **What if young z>9 JWST galaxies hosted massive black holes?**  |
|| Marta Volonteri, <mark>Melanie Habouzit</mark>, Monica Colpi |
|*Appeared on*| *2022-12-12*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| JWST is discovering star forming `candidate' galaxies with photometric redshifts $z>9$ and little attenuation. We model presumptive massive black holes (MBHs) in such galaxies and find that their unobscured emission is fainter than the galaxy starlight in JWST filters, and difficult to be detected via color-color selection, and X-ray and radio observations. Only MBHs overmassive relative to expected galaxy scaling relations, accreting at high Eddington rates, would be detectable. Their discovery would point to the presence of heavy MBH seeds, but care is needed to exclude the existence of lighter seeds as only overmassive MBHs are detectable in this type of galaxies. Conversely, if no overmassive MBHs are hosted in these galaxies, either there are no heavy seeds or they are rare. The most massive/highest redshift candidate galaxies can attain stellar masses in excess of 5e10 Msun by z~6 if they grow along the SFR-mass sequence, and can nurse a MBH growing from ~1e5 Msun up to >3e7 Msun by z~6, to become hosts of some z>6 quasars. Candidate galaxies of log(M_{gal}/Msun)~8 can not grow their putative seeds fast, unless seeds are >1e6 Msun. The number density of the JWST candidate galaxies far outnumbers that of the highest-z quasar hosts and this allows for about only 1 high-redshift quasar every 1000 of these galaxies. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04521-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04521) | **A simulation-based inference pipeline for cosmic shear with the  Kilo-Degree Survey**  |
|| <mark>Kiyam Lin</mark>, Maximilian von Wietersheim-Kramsta, Benjamin Joachimi, Stephen Feeney |
|*Appeared on*| *2022-12-12*|
|*Comments*| *14 pages, 8 figures; submitted to MNRAS*|
|**Abstract**| The standard approach to inference from cosmic large-scale structure data employs summary statistics that are compared to analytic models in a Gaussian likelihood with pre-computed covariance. To overcome the idealising assumptions about the form of the likelihood and the complexity of the data inherent to the standard approach, we investigate simulation-based inference (SBI), which learns the likelihood as a probability density parameterised by a neural network. We construct suites of simulated, exactly Gaussian-distributed data vectors for the most recent Kilo-Degree Survey (KiDS) weak gravitational lensing analysis and demonstrate that SBI recovers the full 12-dimensional KiDS posterior distribution with just under $10^4$ simulations. We optimise the simulation strategy by initially covering the parameter space by a hypercube, followed by batches of actively learnt additional points. The data compression in our SBI implementation is robust to suboptimal choices of fiducial parameter values and of data covariance. Together with a fast simulator, SBI is therefore a competitive and more versatile alternative to standard inference. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04522-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04522) | **A tale of two (or more) $h$'s**  |
|| Samuel Brieden, Héctor Gil-Marín, <mark>Licia Verde</mark> |
|*Appeared on*| *2022-12-12*|
|*Comments*| *42 pages, 12 figures, 3 tables, to be submitted to JCAP. Comments welcome*|
|**Abstract**| We use the large-scale structure galaxy data (LSS) from the BOSS and eBOSS surveys, in combination with abundances information from Big Bang Nucleosynthesis (BBN) to measure two values of the Hubble expansion rate, $H_0=100h\,[{\rm km}\, {\rm s}^{-1}\,{\rm Mpc}^{-1}]$, each of them based on very different physical processes. One is a (traditional) late-time-background measurement based on determining the BAO scale and using BBN abundances on baryons for calibrating its absolute size (BAO+BBN). This method anchors $H_0$ to the (standard) physics of the sound horizon scale at pre-recombination times. The other is a newer early-time based measurement associated with the broadband shape of the power spectrum. This second method anchors $H_0$ to the physics of the matter-radiation equality scale, which also needs BBN information for determining the suppression of baryons in the power spectrum shape (shape+BBN). Within the $\Lambda$CDM model, we find very good consistency among these two $H_0$'s: BAO+BBN (+growth) delivers $H_0=67.42_{-0.94}^{+0.88}$ $(67.37_{-0.95}^{+0.86})$ km s$^{-1}$Mpc$^{-1}$ , whereas the shape+BBN (+growth) delivers $H_0 = 70.1_{-2.1}^{+2.1}$ $(70.1_{-2.1}^{+1.9})$ km s$^{-1}$ Mpc$^{-1}$, where "growth" stands for information from the late-time-perturbations captured by the growth of structure parameter. These are the tightest sound-horizon free $H_0$ constraints from LSS data to date. As a consequence to be viable, any $\Lambda$CDM extension proposed to address the so-called "Hubble tension" needs to modify consistently not only the sound horizon scale physics, but also the matter-radiation equality scale, in such a way that both late- and early-based $H_0$'s return results mutually consistent and consistent with the high $H_0$ value recovered by the standard cosmic distance ladder (distance-redshift relation) determinations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04962-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04962) | **Exploring the single-pulse behaviours of PSR J0628+0909 with FAST**  |
|| J. A. Hsu, et al. -- incl., <mark>K. J. Lee</mark> |
|*Appeared on*| *2022-12-12*|
|*Comments*| *10 pages, 9 figures, accepted by MNRAS*|
|**Abstract**| More than 100 rotating radio transients (RRATs) have been discovered since 2006. However, it is unclear whether RRATs radiate in the nulling states. PSR J0628+0909 has been classified as an RRAT. In this paper, we study the single pulses and integrated pulse profile of PSR J0628+0909 to check whether we can detect pulsed radio emission in the nulling states. We also aim to study the polarization of the RRAT and its relationship to the general pulsar population. We used the Five-hundred-meter Aperture Spherical radio Telescope (FAST) to observe PSR J0628+0909 in the frequency range from 1.0 to 1.5 GHz. We searched for strong single pulses and looked for pulsed emission in the RRAT nulling states. Polarisation profiles, the single-pulse energy distribution, and waiting-time statistics were measured. The Faraday rotation measure and dispersion measure values are updated with the current observation. The single-pulse polarisation behaviours show great diversity, similar to the case of pulsars. Based on the integrated pulse profile and single-pulse energy statistics, we argue that continuous pulsar-like emission exists in addition to the transient-like burst emission for PSR J0628+0909. We find that the pulse waiting time is not correlated with the pulse energy and conclude that the strong transient emission of RRAT is not generated by the energy store-release mechanism. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04990-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04990) | **Spicules in IRIS Mg II Observations: Automated Identification**  |
|| <mark>Vicki L. Herde</mark>, et al. |
|*Appeared on*| *2022-12-12*|
|*Comments*| *17 pages, 9 figures, presented at the AGU Fall 2022 conference, Submitted to AAS Journal*|
|**Abstract**| We have developed an algorithm to identify solar spicules in the first-ever systematic survey of on-disk spicules using exclusively Mg II spectral observations. Using this algorithm we identify 2219 events in three IRIS datasets with unique solar feature targets spanning a total of 300 minutes: 1) an active region, 2) decayed active region/active network, and 3) a coronal hole. We present event statistics and relate occurrence rates to underlying photospheric magnetic field strength. This method identifies spicule event densities and occurrence rates similar to previous studies performed using H{\alpha} and Ca II observations of active regions. Additionally, this study identifies spicule-like events at very low rates at magnetic field intensities below 20 Gauss and increasing significantly between 100-200 Gauss in active regions and above 20 Gauss in coronal holes, which can be used to inform future observation campaigns. This information can be be used to help characterize spicules over their full lifetime, and compliments existing H-{\alpha} spectral capabilities and upcoming Ly-{\alpha} spectral observations on the SNIFS Sounding Rocket. In total, this study presents a method for detecting solar spicules using exclusively Mg II spectra, and provides statistics for spicule occurrence in Mg II wavelengths with respect to magnetic field strength for the purpose of predicting spicule occurrences. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04515-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04515) | **There is No Place Like Home -- Finding Birth Radii of Stars in the Milky  Way**  |
|| Yuxi, et al. -- incl., <mark>Noam Libeskind</mark> |
|*Appeared on*| *2022-12-12*|
|*Comments*| *Under review at Nature Letters, submitted Oct. 19, 2022*|
|**Abstract**| Stars move away from their birth places over time via a process known as radial migration, which blurs chemo-kinematic relations used for reconstructing the Milky Way formation history. One of the ultimate goals of Galactic Archaeology, therefore, is to find stars' birth aggregates in the disk via chemical tagging. Here we show that stellar birth radii can be derived directly from the data with minimum prior assumptions on the Galactic enrichment history. We recover the time evolution of the stellar birth metallicity gradient, $d$[Fe/H]($R$, $\tau$)/$dR$, through its inverse relation to the metallicity range as a function of age today, allowing us to place any star with age and metallicity measurements back to its birthplace, $R_b$. Applying our method to a high-precision large data set of Milky Way disk subgiant stars, we find a steepening of the birth metallicity gradient from 11 to 8 Gyr ago, which coincides with the time of the last major merger, Gaia-Sausage-Enceladus (GSE). This transition appears to play a major role in shaping both the age-metallicity relation and the bimodality in the [$\alpha$/Fe]-[Fe/H] plane. By dissecting the disk into mono-$R_b$ populations, clumps in the low-[$\alpha$/Fe] sequence appear, which are not seen in the total sample and coincide in time with known star-formation bursts. We estimated that the Sun was born at $4.5 \pm 0.4$ kpc from the Galactic center. Our $R_b$ estimates provide the missing piece needed to recover the Milky Way formation history, while the by-product,[Fe/H]$(R$, $\tau)$, can be used as the thus-far missing prior for chemical evolution modeling. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04674-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04674) | **Evolution of Spin Period and Magnetic Field of the Crab Pulsar: Decay of  the Braking Index by the Particle Wind Flow Torque**  |
|| <mark>Cheng-Min Zhang</mark>, et al. -- incl., <mark>Di Li</mark>, <mark>Jian-Wei Zhang</mark> |
|*Appeared on*| *2022-12-12*|
|*Comments*| *21pages, 3 figures, 2 tables, and 5 appendices, published in Universe*|
|**Abstract**| The evolutions of a neutron star's rotation and magnetic field (B-field) have remained unsolved puzzles for over half a century. We ascribe the rotational braking torques of pulsar to both components, the standard magnetic dipole radiation (MDR) and particle wind flow ( MDR + Wind, hereafter named MDRW), which we apply to the Crab pulsar (B0531 + 21), the only source with a known age and long-term continuous monitoring by radio telescope. Based on the above presumed simple spin-down torques, we obtain the exact analytic solution on the rotation evolution of the Crab pulsar, together with the related outcomes as described below: (1) unlike the constant characteristic B-field suggested by the MDR model, this value for the Crab pulsar increases by a hundred times in 50~kyr while its real B-field has no change; (2) the rotational braking index evolves from $\sim$3 to 1 in the long-term, however, it drops from 2.51 to 2.50 in $\sim$45 years at the present stage, while the particle flow contributes approximately 25% of the total rotational energy loss rate; (3) strikingly, the characteristic age has the maximum limit of $\sim$10 kyr, meaning that it is not always a good indicator of real age. Furthermore, we discussed the evolutionary path of the Crab pulsar from the MDR to the wind domination by comparing it with the possible wind braking candidate pulsar PSR J1734-3333. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04568-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04568) | **Spectroscopy of four metal-poor galaxies beyond redshift ten**  |
|| Emma Curtis-Lake, et al. -- incl., <mark>Hans-Walter Rix</mark>, <mark>Anna de Graaff</mark>, <mark>Jakob M. Helton</mark> |
|*Appeared on*| *2022-12-12*|
|*Comments*| *32 pages, 9 figures, Submitted*|
|**Abstract**| Finding and characterising the first galaxies that illuminated the early Universe at cosmic dawn is pivotal to understand the physical conditions and the processes that led to the formation of the first stars. In the first few months of operations, imaging from the James Webb Space Telescope (JWST) have been used to identify tens of candidates of galaxies at redshift (z) greater than 10, less than 450 million years after the Big Bang. However, none of these candidates has yet been confirmed spectroscopically, leaving open the possibility that they are actually low-redshift interlopers. Here we present spectroscopic confirmation and analysis of four galaxies unambiguously detected at redshift 10.3<z<13.2, previously selected from NIRCam imaging. The spectra reveal that these primeval galaxies are extremely metal poor, have masses between 10^7 and a few times 10^8 solar masses, and young ages. The damping wings that shape the continuum close to the Lyman edge are consistent with a fully neutral intergalactic medium at this epoch. These findings demonstrate the rapid emergence of the first generations of galaxies at cosmic dawn. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error expected string or bytes-like object</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.04710.md
    + _build/html/tmp_2212.04710/./Figures/LGAL_LAGN_Mstar_MBH_app_z9.png
    + _build/html/tmp_2212.04710/./Figures/MBH01e_4_varfedd.png
    + _build/html/tmp_2212.04710/./Figures/MBH01e_4_varfedd_SN3e9.png
    + _build/html/tmp_2212.04710/./Figures/MBH0opt_varfedd.png
    + _build/html/tmp_2212.04710/./Figures/MBH0opt_fedd05.png
    + _build/html/tmp_2212.04710/./Figures/LGAL_LAGN_Mstar_MBH_zoom_z9.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\aj}{AJ}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{ARA\&A}$
$\newcommand{\apj}{ApJ}$
$\newcommand{\apjl}{ApJ}$
$\newcommand{\apjs}{ApJS}$
$\newcommand{\ao}{Appl.~Opt.}$
$\newcommand{\apss}{Ap\&SS}$
$\newcommand{\aap}{A\&A}$
$\newcommand{\aapr}{A\&AR}$
$\newcommand{\aaps}{A\&AS}$
$\newcommand{\azh}{AZh}$
$\newcommand{\baas}{BAAS}$
$\newcommand{\caa}{Chinese Astron. Astrophys.}$
$\newcommand{\cjaa}{Chinese J. Astron. Astrophys.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\jcap}{J. Cosmology Astropart. Phys.}$
$\newcommand{\jrasc}{JRASC}$
$\newcommand{\memras}{MmRAS}$
$\newcommand{\mnras}{MNRAS}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New A Rev.}$
$\newcommand{\pra}{Phys.~Rev.~A}$
$\newcommand{\prb}{Phys.~Rev.~B}$
$\newcommand{\prc}{Phys.~Rev.~C}$
$\newcommand{\prd}{Phys.~Rev.~D}$
$\newcommand{\pre}{Phys.~Rev.~E}$
$\newcommand{\prl}{Phys.~Rev.~Lett.}$
$\newcommand{\pasa}{PASA}$
$\newcommand{\pasp}{PASP}$
$\newcommand{\pasj}{PASJ}$
$\newcommand{\qjras}{QJRAS}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\skytel}{S\&T}$
$\newcommand{\solphys}{Sol.~Phys.}$
$\newcommand{\sovast}{Soviet~Ast.}$
$\newcommand{\ssr}{Space~Sci.~Rev.}$
$\newcommand{\zap}{ZAp}$
$\newcommand{\nat}{Nature}$
$\newcommand{\iaucirc}{IAU~Circ.}$
$\newcommand{\aplett}{Astrophys.~Lett.}$
$\newcommand{\apspr}{Astrophys.~Space~Phys.~Res.}$
$\newcommand{\bain}{Bull.~Astron.~Inst.~Netherlands}$
$\newcommand{\fcp}{Fund.~Cosmic~Phys.}$
$\newcommand{\gca}{Geochim.~Cosmochim.~Acta}$
$\newcommand{\grl}{Geophys.~Res.~Lett.}$
$\newcommand{\jcp}{J.~Chem.~Phys.}$
$\newcommand{\jgr}{J.~Geophys.~Res.}$
$\newcommand{\jqsrt}{J.~Quant.~Spec.~Radiat.~Transf.}$
$\newcommand{\memsai}{Mem.~Soc.~Astron.~Italiana}$
$\newcommand{\nphysa}{Nucl.~Phys.~A}$
$\newcommand{\physrep}{Phys.~Rep.}$
$\newcommand{\physscr}{Phys.~Scr}$
$\newcommand{\planss}{Planet.~Space~Sci.}$
$\newcommand{\procspie}{Proc.~SPIE}$
$\newcommand{\edd}{{\rm Edd}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\msun}{ {\rm M_\odot}}$
$\title[MBHs in z>9 JWST galaxies ]{What if young z>9 JWST galaxies hosted massive black holes?}$
$\author[Volonteri et al.]{Marta Volonteri,^{1}\thanks{E-mail: martav@iap.fr (MV)}$
$M\'elanie Habouzit,^{2,3}$
$Monica Colpi^{4,5}$
$\^{1}Institut d'Astrophysique de Paris, Sorbonne Universit\'e, CNRS, UMR 7095, 98 bis bd Arago, 75014 Paris, France\^{2}Zentrum f\"ur Astronomie der Universit\"at Heidelberg, ITA, Albert-Ueberle-Str. 2, D-69120 Heidelberg, Germany\^{3}Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany\^{4}Dipartimento di Fisica G. Occhialini, Universita di Milano-Bicocca, Piazza della Scienza 3, I-20126 Milano, Italy\^{5}INFN, Sezione di Milano-Bicocca, Piazza della Scienza 3, I-20126 Milano, Italy\}$
$\begin{document}$
$\maketitle$
$\begin{abstract}$
$JWST is discovering star forming `candidate' galaxies with photometric redshifts z>9 and little attenuation. We model presumptive massive black holes (MBHs) in such galaxies and find that their unobscured emission is fainter than the galaxy starlight in JWST filters, and difficult to be detected via color-color selection, and X-ray and radio observations. Only MBHs  overmassive relative to expected galaxy scaling relations, accreting at high Eddington rates, would be detectable. Their discovery would point to the presence of heavy MBH seeds, but care is needed to exclude the existence of lighter seeds  as only overmassive MBHs are detectable in this type of galaxies.  Conversely, if no overmassive MBHs are hosted in these galaxies, either there are no heavy seeds or they are rare. The most massive/highest redshift candidate galaxies can attain stellar masses in excess of 5\times 10^{10}\msun by z\sim 6 if they grow along the SFR-mass sequence, and can nurse a MBH growing from \sim 10^5 \msun up to >3\times 10^7\msun by z\sim 6, to become hosts of some z > 6 quasars. Candidate galaxies of  \log( M_{\rm gal}/\msun)\sim 8 can not grow their putative seeds fast, unless seeds are \gtrsim 10^6\msun.  The number density of the JWST candidate galaxies far outnumbers that of the highest-z quasar hosts and this allows for about only 1 high-redshift quasar every 1000 of these galaxies.$
$\end{abstract}$
$\begin{keywords}$
$galaxies: high-redshift  - quasars: supermassive black holes -  galaxies: evolution - galaxies: active$
$\end{keywords}$
$\n\end{document}\end{equation}}$
$\newcommand{\eeq}{\end{equation}}$
$\newcommand{\msun}{ {\rm M_\odot}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\aj$}{AJ}$
$\newcommand{$\actaa$}{Acta Astron.}$
$\newcommand{$\araa$}{ARA\&A}$
$\newcommand{$\apj$}{ApJ}$
$\newcommand{$\apj$l}{ApJ}$
$\newcommand{$\apj$s}{ApJS}$
$\newcommand{$\ao$}{Appl.~Opt.}$
$\newcommand{$\apss$}{Ap\&SS}$
$\newcommand{$\aap$}{A\&A}$
$\newcommand{$\aap$r}{A\&AR}$
$\newcommand{$\aap$s}{A\&AS}$
$\newcommand{$\azh$}{AZh}$
$\newcommand{$\baas$}{BAAS}$
$\newcommand{$\caa$}{Chinese Astron. Astrophys.}$
$\newcommand{$\cjaa$}{Chinese J. Astron. Astrophys.}$
$\newcommand{$\icarus$}{Icarus}$
$\newcommand{$\jcap$}{J. Cosmology Astropart. Phys.}$
$\newcommand{$\jrasc$}{JRASC}$
$\newcommand{$\memras$}{MmRAS}$
$\newcommand{$\mnras$}{MNRAS}$
$\newcommand{$\na$}{New Astron.}$
$\newcommand{$\na$r}{New A Rev.}$
$\newcommand{$\pra$}{Phys.~Rev.~A}$
$\newcommand{$\prb$}{Phys.~Rev.~B}$
$\newcommand{$\prc$}{Phys.~Rev.~C}$
$\newcommand{$\prd$}{Phys.~Rev.~D}$
$\newcommand{$\pre$}{Phys.~Rev.~E}$
$\newcommand{$\prl$}{Phys.~Rev.~Lett.}$
$\newcommand{$\pasa$}{PASA}$
$\newcommand{$\pasp$}{PASP}$
$\newcommand{$\pasj$}{PASJ}$
$\newcommand{$\qjras$}{QJRAS}$
$\newcommand{$\rmxaa$}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{$\skytel$}{S\&T}$
$\newcommand{$\solphys$}{Sol.~Phys.}$
$\newcommand{$\sovast$}{Soviet~Ast.}$
$\newcommand{$\ssr$}{Space~Sci.~Rev.}$
$\newcommand{$\zap$}{ZAp}$
$\newcommand{$\na$t}{Nature}$
$\newcommand{$\iaucirc$}{IAU~Circ.}$
$\newcommand{$\aplett$}{Astrophys.~Lett.}$
$\newcommand{$\apspr$}{Astrophys.~Space~Phys.~Res.}$
$\newcommand{$\bain$}{Bull.~Astron.~Inst.~Netherlands}$
$\newcommand{$\fcp$}{Fund.~Cosmic~Phys.}$
$\newcommand{$\gca$}{Geochim.~Cosmochim.~Acta}$
$\newcommand{$\grl$}{Geophys.~Res.~Lett.}$
$\newcommand{$\jcp$}{J.~Chem.~Phys.}$
$\newcommand{$\jgr$}{J.~Geophys.~Res.}$
$\newcommand{$\jqsrt$}{J.~Quant.~Spec.~Radiat.~Transf.}$
$\newcommand{$\memsai$}{Mem.~Soc.~Astron.~Italiana}$
$\newcommand{$\nphysa$}{Nucl.~Phys.~A}$
$\newcommand{$\physrep$}{Phys.~Rep.}$
$\newcommand{$\physscr$}{Phys.~Scr}$
$\newcommand{$\planss$}{Planet.~Space~Sci.}$
$\newcommand{$\procspie$}{Proc.~SPIE}$
$\newcommand{$\edd$}{{\rm Edd}}$
$\newcommand{\beq}{\begin{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{$\msun$}{ {\rm M_\odot}}$
$\title[MBHs in z>9 JWST galaxies ]{What if young z>9 JWST galaxies hosted massive black holes?}$
$\author[Volonteri et al.]{Marta Volonteri,^{1}\thanks{E-mail: martav@iap.fr (MV)}$
$M\'elanie Habouzit,^{2,3}$
$Monica Colpi^{4,5}$
$\^{1}Institut d'Astrophysique de Paris, Sorbonne Universit\'e, CNRS, UMR 7095, 98 bis bd Arago, 75014 Paris, France\^{2}Zentrum f\"ur Astronomie der Universit\"at Heidelberg, ITA, Albert-Ueberle-Str. 2, D-69120 Heidelberg, Germany\^{3}Max-Planck-Institut f\"ur Astronomie, K\"onigstuhl 17, D-69117 Heidelberg, Germany\^{4}Dipartimento di Fisica G. Occhialini, Universita di Milano-Bicocca, Piazza della Scienza 3, I-20126 Milano, Italy\^{5}INFN, Sezione di Milano-Bicocca, Piazza della Scienza 3, I-20126 Milano, Italy\}$
$\begin{document}$
$\maketitle$
$\begin{abstract}$
$JWST is discovering star forming `candidate' galaxies with photometric redshifts z>9 and little attenuation. We model presumptive massive black holes (MBHs) in such galaxies and find that their unobscured emission is fainter than the galaxy starlight in JWST filters, and difficult to be detected via color-color selection, and X-ray and radio observations. Only MBHs  overmassive relative to expected galaxy scaling relations, accreting at high Eddington rates, would be detectable. Their discovery would point to the presence of heavy MBH seeds, but care is needed to exclude the existence of lighter seeds  as only overmassive MBHs are detectable in this type of galaxies.  Conversely, if no overmassive MBHs are hosted in these galaxies, either there are no heavy seeds or they are rare. The most massive/highest redshift candidate galaxies can attain stellar masses in excess of 5\times 10^{10}$\msun$ by z\sim 6 if they grow along the SFR-mass sequence, and can nurse a MBH growing from \sim 10^5 $\msun$ up to >3\times 10^7$\msun$ by z\sim 6, to become hosts of some z > 6 quasars. Candidate galaxies of  \log( M_{\rm gal}/$\msun$)\sim 8 can not grow their putative seeds fast, unless seeds are \gtrsim 10^6$\msun$.  The number density of the JWST candidate galaxies far outnumbers that of the highest-z quasar hosts and this allows for about only 1 high-redshift quasar every 1000 of these galaxies.$
$\end{abstract}$
$\begin{keywords}$
$galaxies: high-redshift  - quasars: supermassive black holes -  galaxies: evolution - galaxies: active$
$\end{keywords}$
$\n\end{document}\end{equation}}$
$\newcommand{$\eeq$}{\end{equation}}$
$\newcommand{$\msun$}{ {\rm M_\odot}}$</div>



<div id="title">

#  JWST galaxies hosted massive black holes?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.04710-b31b1b.svg)](https://arxiv.org/abs/2212.04710)<mark>Appeared on: 2022-12-12</mark> - _Submitted to MNRAS_

</div>
<div id="authors">

Marta Volonteri, <mark><mark>Melanie Habouzit</mark></mark>, Monica Colpi

</div>
<div id="abstract">

**Abstract:** JWST is discovering star forming `candidate' galaxies with photometric redshifts$z>9$and little attenuation. We model presumptive massive black holes (MBHs) in such galaxies and find that their unobscured emission is fainter than the galaxy starlight in JWST filters, and difficult to be detected via color-color selection, and X-ray and radio observations. Only MBHs  overmassive relative to expected galaxy scaling relations, accreting at high Eddington rates, would be detectable. Their discovery would point to the presence of heavy MBH seeds, but care is needed to exclude the existence of lighter seeds  as only overmassive MBHs are detectable in this type of galaxies.  Conversely, if no overmassive MBHs are hosted in these galaxies, either there are no heavy seeds or they are rare. The most massive/highest redshift candidate galaxies can attain stellar masses in excess of$5\times 10^{10}$\msun$$by$z\sim 6$if they grow along the SFR-mass sequence, and can nurse a MBH growing from$\sim 10^5 $\msun$$up to$>3\times 10^7$\msun$$by$z\sim 6$, to become hosts of some$z > 6$quasars. Candidate galaxies of$\log( M_{\rm gal}/$\msun$)\sim 8$can not grow their putative seeds fast, unless seeds are$\gtrsim 10^6$\msun$$.  The number density of the JWST candidate galaxies far outnumbers that of the highest-$z$quasar hosts and this allows for about only 1 high-redshift quasar every 1000 of these galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2212.04710/./Figures/LGAL_LAGN_Mstar_MBH_app_z9.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of AGN and galaxy luminosity using our models: apparent magnitude of AGN and galaxies.
    Triangles show the AGN magnitudes corresponding to MBHs following different scaling relations with the host galaxies, and accreting at fractions of the Eddington limit. Green: MBHs with mass $\log(M_{\rm gal}/$\msun$)-5$; turquoise: MBHs with mass $\log(M_{\rm gal}/$\msun$)-4$; blue: MBHs with mass $\log(M_{\rm gal}/$\msun$)-3$; slate grey: MBHs with mass $\log(M_{\rm gal}/$\msun$)-2$; grey: MBHs with mass $\log(M_{\rm gal}/$\msun$)-1$. The size of the symbol scales with the Eddington ratio: small for $\log(f_{\rm Edd})=-2$, medium for $\log(f_{\rm Edd})=-1$, large for $\log(f_{\rm Edd})=0$.
    Red dots: F356W or F444W  from observational references (only a fraction of the candidates have published photometry, therefore some of the sources are not shown). Photometric errors are not shown for clarity but they are less than 10 per cent. Orange dots: galaxy apparent magnitude from our models. Galaxies are shown at the mass corresponding to $\log(M_{\rm gal}/$\msun$)-4$. The simple galaxy model we use produces reasonable results. When comparing AGN and galaxy magnitudes we see that AGN can be brighter than the host only when they are overmassive with respect to the nominal relation and they have high Eddington ratios. (*fig:compareL*)

</div>
<div id="div_fig2">

<img src="tmp_2212.04710/./Figures/MBH01e_4_varfedd.png" alt="Fig7.1" width="25%"/><img src="tmp_2212.04710/./Figures/MBH01e_4_varfedd_SN3e9.png" alt="Fig7.2" width="25%"/><img src="tmp_2212.04710/./Figures/MBH0opt_varfedd.png" alt="Fig7.3" width="25%"/><img src="tmp_2212.04710/./Figures/MBH0opt_fedd05.png" alt="Fig7.4" width="25%"/>

**Figure 7. -** Examples of MBH and galaxy evolution, taking as initial conditions the galaxies in Table 1 and making different assumptions for the MBHs they host (top: $\log(M_{\rm BH}/$\msun$)=\log(M_{\rm gal}/$\msun$)-4$; bottom:  $\log(M_{\rm BH}/$\msun$)=\max(5,\log(M_{\rm gal}/$\msun$)-3)$), effect of SNae (top-left and bottom: $M_{\rm SN}=10^9 $\msun$$; top-right: $M_{\rm SN}=3\times10^9 $\msun$$), and their growth rate (top, bottom-left: scaling with SFR; bottom-right: $f_{\rm Edd}=0.5$). Galaxies are always assumed to grow along the SFR-galaxy mass sequence. The thicker curves highlight some example galaxies: SMACS\_z10b, SMACS\_z16b, S5-z17-1, GL-z12. In all panels the bottom region shows the ratio of MBH to galaxy mass, without highlighting specific galaxies. (*fig:growth*)

</div>
<div id="div_fig3">

<img src="tmp_2212.04710/./Figures/LGAL_LAGN_Mstar_MBH_zoom_z9.png" alt="Fig2" width="100%"/>

**Figure 2. -** Difference between the AGN and galaxy magnitude (shown only for magnitude differences less than $\pm2$). Colors as in Fig. \ref{fig:compareL} and the symbol size also scales with Eddington ratio as described in that Figure. The grey shaded area shows the region where the AGN is fainter than the stellar component.
At most JWST wavelengths only MBHs with mass larger than $\log(M_{\rm gal}/$\msun$)-2$ can outshine their host galaxy, whereas MBHs with mass $\log(M_{\rm gal}/$\msun$)-2$ and $\log(f_{\rm Edd})=0$ can outshine the galaxy at the longest JWST wavelengths. MBHs on the nominal relation are always fainter than their host galaxy.  (*fig:compareLzoom*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

131  publications in the last 7 days.
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers